In [1]:
from rchess import Board
import numpy as np

In [5]:
board = Board()
print(board.to_string())


  +---+---+---+---+---+---+---+---+
  | r | n | b | q | k | b | n | r | 8
  +---+---+---+---+---+---+---+---+
  | p | p | p | p | p | p | p | p | 7
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 6
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 5
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 4
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 3
  +---+---+---+---+---+---+---+---+
  | P | P | P | P | P | P | P | P | 2
  +---+---+---+---+---+---+---+---+
  | R | N | B | Q | K | B | N | R | 1
  +---+---+---+---+---+---+---+---+
    a   b   c   d   e   f   g   h

Fen: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
Key: 8cefbf3a77078a08


In [7]:
from utils import convert_u64_to_np

def test_model(input): # This is trt_func in prod
    shape = input.shape
    return {
        "policy_head": np.random.rand(shape[0], 1858),
        "value_head": np.random.rand(shape[0], 1)
    }

def predict_fn(trt_func, input):
    predictions = trt_func(input)
    policy_logits = predictions["policy_head"]
    value = predictions["value_head"]
    return value, policy_logits
    
def make_predictions(data): # Not sure yet what data is
    input = convert_u64_to_np(data)
    value, policy_logits = predict_fn(test_model, input)
    return value, policy_logits

board = Board()
history, _ = board.history(False)
make_predictions(history)

(array([[0.16022421]]),
 array([[0.72136392, 0.57083357, 0.47040038, ..., 0.77928933, 0.24787217,
         0.51568062]]))

In [1]:
from model import load_as_trt_model
import numpy as np
trt_func, model = load_as_trt_model()


from rchess import Board
import time

2024-12-27 21:29:10.493840: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-27 21:29:11.916788: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5865 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 SUPER, pci bus id: 0000:26:00.0, compute capability: 7.5
2024-12-27 21:29:12.516727: E tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:87] DefaultLogger 3: [runtime.cpp::~Runtime::346] Error Code 3: API Usage Error (Parameter check failed at: runtime/rt/runtime.cpp::~Runtime::346, condition: mEngineCounter.use_count() == 1. Destroying a runtime before destroying deserialized engines created by the runtime leads to undefined behavior.
)


In [2]:
board = Board()

In [3]:
from mcts import find_best_move, Node

times = []
for i in range(10):
    board = Board()
    root = None
    for i in range(10):
        t1 = time.time()
        move, root, child_visits = find_best_move(board, root, trt_func, 800)
        t2 = time.time()
        times.append(t2 - t1)
        board.push_uci(move)
        terminal, winner = board.terminal()
        if terminal:
            print("Game over, winner is", winner)
        root = root[move]
print("Average time", np.mean(times))


Average time 0.11894978284835815


In [ ]:
import pstats, cProfile
from mcts import find_best_move
board = Board()
cProfile.runctx("find_best_move(board, None, trt_func, 800)", globals(), locals(), "Profile.prof")

s = pstats.Stats("Profile.prof")
s.strip_dirs().sort_stats("time").print_stats()

In [6]:
print(board.to_string())


  +---+---+---+---+---+---+---+---+
  | r |   | b | q | k | b |   | r | 8
  +---+---+---+---+---+---+---+---+
  | p | p | p | p | p | p | p | p | 7
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 6
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 5
  +---+---+---+---+---+---+---+---+
  |   |   |   |   | n |   |   |   | 4
  +---+---+---+---+---+---+---+---+
  | P |   | N |   |   | N |   |   | 3
  +---+---+---+---+---+---+---+---+
  |   | P | Q | P | P | P | P | P | 2
  +---+---+---+---+---+---+---+---+
  | R |   | B | K |   | B |   | R | 1
  +---+---+---+---+---+---+---+---+
    a   b   c   d   e   f   g   h

Fen: r1bqkb1r/pppppppp/8/8/4n3/P1N2N2/1PQPPPPP/R1BK1B1R w kq - 3 6
Key: ef69a98e93ec1388


In [2]:
from mcts import SelfplayGame
times = []
for i in range(10):
    game = SelfplayGame()
    for i in range(10):
        t1 = time.time()
        move, child_visits = game.simulate(trt_func, 800)
        t2 = time.time()
        times.append(t2 - t1)
        game.make_move(move)
        terminal, winner = game.board.terminal()
        if terminal:
            print("Game over, winner is", winner)
print("Average time", np.mean(times))

Average time 0.31473344564437866


In [27]:
t1 = time.time()
move, child_visits = game.simulate(trt_func, 800)
t2 = time.time()
print(t2-t1)
game.make_move(move)
terminal, winner = game.board.terminal()
if terminal:
    print("Game over, winner is", winner)

0.4044685363769531


In [16]:
game = SelfplayGame()
t1 = time.time()
while True:
    move, child_visits = game.simulate(trt_func, 800)
    game.make_move(move)
    terminal, winner = game.board.terminal()
    if terminal:
        break
if winner == 1:
    print("White wins")
elif winner == -1:
    print("Black wins")
else:
    print("Draw")
t2 = time.time()
print(game.board.to_string())
print(t2-t1)
print(move)


Draw

  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 8
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 7
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 6
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 5
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 4
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   | Q |   |   | 3
  +---+---+---+---+---+---+---+---+
  |   |   | K |   |   |   |   |   | 2
  +---+---+---+---+---+---+---+---+
  |   |   |   |   | k |   |   |   | 1
  +---+---+---+---+---+---+---+---+
    a   b   c   d   e   f   g   h

Fen: 8/8/8/8/8/5Q2/2K5/4k3 b - - 66 199
Key: d554af9f066be6d5
134.74590635299683
b1c2


In [4]:
import numpy as np
import timeit
import time
from math import sqrt, log

class Node:
    @property
    def Q(self):
        return self.W / self.N if self.N > 0 else 0.0

    def __init__(self):
        self.children = {}
        self.vloss = 0
        self.to_play = None

        # Values for MCTS
        self.N = 0
        self.W = 0.0
        self.P = 0.0

    def __getitem__(self, move: str):
        return self.children[move]
    
    def __setitem__(self, move: str, node):
        self.children[move] = node

    def is_leaf(self):
        return not self.children
    
# Create a dummy node with 30 fake children
dummy_node = Node()
dummy_node.N = 100  # Set parent visit count

for i in range(30):
    child = Node()
    child.N = np.random.randint(1, 100)  # Random visit count for each child
    child.W = np.random.rand() * 100  # Random total value for each child
    child.P = np.random.rand()  # Random prior probability for each child
    dummy_node.children[f"move_{i}"] = child

In [7]:
def select_child(node: Node, pb_c_base: float, pb_c_init: float, pb_c_factor: float, fpu: float):
    bestucb = -np.inf
    bestmove = None
    bestchild = None

    parent_N_sqrt = sqrt(node.N + 1)
    pb_c = log((node.N + pb_c_base + 1) / pb_c_base) * pb_c_factor + pb_c_init

    for move, child in node.children.items():
        if child.N == 0:
            puct = child.Q + pb_c * child.P * parent_N_sqrt / (child.N + 1)
        else:
            puct = fpu
        if puct > bestucb:
            bestucb = puct
            bestmove = move
            bestchild = child

    return bestmove, bestchild

# Create a dummy node with 30 fake children
dummy_node = Node()
dummy_node.N = 100  # Set parent visit count

for i in range(30):
    child = Node()
    child.N = np.random.randint(1, 100)  # Random visit count for each child
    child.W = np.random.rand() * 100  # Random total value for each child
    child.P = np.random.rand()  # Random prior probability for each child
    dummy_node.children[f"move_{i}"] = child

# Benchmark the select_child function
pb_c_base = 19652
pb_c_init = 1.25
pb_c_factor = 1.0
fpu = 0.0#bestmove, bestchild = select_child(dummy_node, pb_c_base, pb_c_init, pb_c_factor, fpu)

num_runs = 1000000
# Benchmark the select_child function using timeit
benchmark_time = timeit.timeit(
    "select_child(dummy_node, pb_c_base, pb_c_init, pb_c_factor, fpu)",
    globals=globals(),
    number=num_runs
)

print(f"Average time taken over {num_runs} runs: {benchmark_time / num_runs} seconds")
print(f"Total time taken: {benchmark_time} seconds")

Average time taken over 1000000 runs: 1.9494292420004056e-06 seconds
Total time taken: 1.9494292420004058 seconds


In [6]:
from mcts.c import select_child

# Create a dummy node with 30 fake children
dummy_node = Node()
dummy_node.N = 100  # Set parent visit count

for i in range(30):
    child = Node()
    child.N = np.random.randint(1, 100)  # Random visit count for each child
    child.W = np.random.rand() * 100  # Random total value for each child
    child.P = np.random.rand()  # Random prior probability for each child
    dummy_node.children[f"move_{i}"] = child

# Benchmark the select_child function
pb_c_base = 19652
pb_c_init = 1.25
pb_c_factor = 1.0
fpu = 0.0#bestmove, bestchild = select_child(dummy_node, pb_c_base, pb_c_init, pb_c_factor, fpu)

num_runs = 1000000
# Benchmark the select_child function using timeit
benchmark_time = timeit.timeit(
    "select_child(dummy_node, pb_c_base, pb_c_init, pb_c_factor, fpu)",
    globals=globals(),
    number=num_runs
)

print(f"Average time taken over {num_runs} runs: {benchmark_time / num_runs} seconds")
print(f"Total time taken: {benchmark_time} seconds")

Average time taken over 1000000 runs: 5.751467379999667e-06 seconds
Total time taken: 5.751467379999667 seconds


In [4]:
# Update and remove vloss
def backup(search_path, value):
    for node in reversed(search_path):
        node.N = node.N - node.vloss + 1
        node.W = node.W + node.vloss + value
        node.vloss = 0
        value = flip_value(value)

# Create a fake search path with at least 30 elements
search_path = [Node() for _ in range(30)]
value = np.random.rand()

# Benchmark the backup function using timeit
backup_time = timeit.timeit(
    "backup(search_path, value)",
    globals=globals(),
    number=num_runs
)

print(f"Average time taken over {num_runs} runs: {backup_time / num_runs} seconds")
print(f"Total time taken: {backup_time} seconds")

Average time taken over 1000000 runs: 6.049426947000029e-06 seconds
Total time taken: 6.049426947000029 seconds


In [6]:
from mcts.c import backup

# Create a fake search path with at least 30 elements
search_path = [Node() for _ in range(30)]
value = np.random.rand()

# Benchmark the backup function using timeit
backup_time = timeit.timeit(
    "backup(search_path, value)",
    globals=globals(),
    number=num_runs
)

print(f"Average time taken over {num_runs} runs: {backup_time / num_runs} seconds")
print(f"Total time taken: {backup_time} seconds")

Average time taken over 1000000 runs: 4.521437020999656e-06 seconds
Total time taken: 4.521437020999656 seconds


In [7]:
import timeit

# Sample list
sample_list = list(range(1000))

# Method 1: Using reversed()
def traverse_reversed(lst):
    for item in reversed(lst):
        pass

# Method 2: Using slicing
def traverse_slicing(lst):
    for item in lst[::-1]:
        pass

# Method 3: Using a for loop with range
def traverse_range(lst):
    for i in range(len(lst)-1, -1, -1):
        item = lst[i]

# Number of runs for benchmarking
num_runs = 100000

# Benchmark each method
time_reversed = timeit.timeit('traverse_reversed(sample_list)', globals=globals(), number=num_runs)
time_slicing = timeit.timeit('traverse_slicing(sample_list)', globals=globals(), number=num_runs)
time_range = timeit.timeit('traverse_range(sample_list)', globals=globals(), number=num_runs)

print(f"Average time taken by reversed() over {num_runs} runs: {time_reversed / num_runs} seconds")
print(f"Average time taken by slicing over {num_runs} runs: {time_slicing / num_runs} seconds")
print(f"Average time taken by range over {num_runs} runs: {time_range / num_runs} seconds")

Average time taken by reversed() over 100000 runs: 6.639731540008142e-06 seconds
Average time taken by slicing over 100000 runs: 7.250656179994621e-06 seconds
Average time taken by range over 100000 runs: 2.4007662889998754e-05 seconds


In [25]:
import timeit

# Define the node and policy_np for testing
node = Node()
policy_np = np.random.rand(30)

# Create dummy children for the node
for i in range(30):
    child = Node()
    node.children[f"move_{i}"] = child

# Define the first loop
def loop1():
    for action, move_uci in enumerate(node.children.keys()):
        node.children[move_uci].P = policy_np[action]

# Define the second loop
def loop2():
    for p, child in zip(policy_np, node.children.values()):
        child.P = p

def loop3():
    i = 0
    for child in node.children.values():
        child.P = policy_np[i]
        i += 1

# Number of runs for benchmarking
num_runs = 100000

# Benchmark each loop
time_loop1 = timeit.timeit('loop1()', globals=globals(), number=num_runs)
time_loop2 = timeit.timeit('loop2()', globals=globals(), number=num_runs)
time_loop3 = timeit.timeit('loop3()', globals=globals(), number=num_runs)

print(f"Average time taken by loop1 over {num_runs} runs: {time_loop1 / num_runs} seconds")
print(f"Average time taken by loop2 over {num_runs} runs: {time_loop2 / num_runs} seconds")
print(f"Average time taken by loop3 over {num_runs} runs: {time_loop3 / num_runs} seconds")

Average time taken by loop1 over 100000 runs: 3.7831326599916793e-06 seconds
Average time taken by loop2 over 100000 runs: 3.0510011699971073e-06 seconds
Average time taken by loop3 over 100000 runs: 3.2348851200003993e-06 seconds
